In [2]:
import pandas as pd
import datetime

#FUNCTIONS
def get_data():
    import requests
    nba_url = "https://stats.nba.com/stats/leaguegamelog?Counter=1000&DateFrom=&DateTo=&Direction=DESC&LeagueID=00&PlayerOrTeam=P&Season=2018-19&SeasonType=Regular+Season&Sorter=DATE"
    this_user_agent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
    re = requests.get(nba_url, headers=this_user_agent)
    print(re.status_code)
    import json
    nba_json = json.loads(re.text)
    return nba_json

def make_json_df(nba_json):
    headers = nba_json['resultSets'][0]['headers']
    data = nba_json['resultSets'][0]['rowSet']
    df = pd.DataFrame(data, columns=headers)
    return df

def clean_df(df):
    df = df.drop(columns=['SEASON_ID', 'PLAYER_ID', 'TEAM_ID','GAME_ID','VIDEO_AVAILABLE'])
    df['GAME_DATE'] = df['GAME_DATE'].astype(str)
    return df

def get_time_ellapsed(str_date):
    ymd = str_date.split('-')
    y = int(ymd[0])
    m = int(ymd[1])
    d = int(ymd[2])
    then = datetime.datetime(y,m,d)
    rn = datetime.datetime.now()
    delta = rn - then
    return delta.days

def make_days_since_col(df):
    dates = df['GAME_DATE']
    days_since_arr = []
    for i in dates:
        days_since_arr.append(get_time_ellapsed(i))
    df['DAYS_SINCE_RN'] = days_since_arr
    return df

def get_team_df(team, df):
    team_df = df[df['TEAM_ABBREVIATION']==team]
    return team_df

def get_player_df(player, df):
    player_df = df[df['PLAYER_NAME']==player]
    return player_df

def get_team_json(team_abbr):
    nba_json = get_data()
    df = clean_df(make_json_df(nba_json))
    df = make_days_since_col(df)
    team_df = get_team_df(team_abbr, df)
    team_json = team_df.to_json()
    return team_json

def predict_stat(player, stat, df):
    player_df = get_player_df(player, df)
    sum_days = 0
    for num_days in player_df['DAYS_SINCE_RN']:
        sum_days += num_days
    sum_days
    importances = []
    for num_days in player_df['DAYS_SINCE_RN']:
        importance = ((sum_days - num_days)/sum_days)
        importances.append(importance**3)
    stat_ser = player_df[stat]
    stats = []
    for stat in stat_ser:
        stats.append(int(stat))
    scores = []
    for i in range(len(stats)):
        score = importances[i]*stats[i]
        scores.append(score)
    sum_importance = 0
    for imp in importances:
        sum_importance += imp
    p_stat = sum(scores)/sum_importance
    return round(p_stat, 3)

In [20]:
def predict(team):
    nba_json = get_data()
    df = clean_df(make_json_df(nba_json))
    df = make_days_since_col(df)
    team_df = get_team_df(team, df)
    players = team_df['PLAYER_NAME'].unique()
    
    p_json_out = []
    for player in players:
        p_json_out.append({
            'NAME': player,
            'PTS': predict_stat(player, 'PTS', team_df),
            'AST': predict_stat(player, 'AST', team_df),
            'REB': predict_stat(player, 'REB', team_df)
        })
        
    return p_json_out

In [21]:
lal_json = predict('LAL')
lal_json



200


[{'name': 'Tyson Chandler', 'PTS': 3.97, 'AST': 0.623, 'REB': 6.992},
 {'name': 'Michael Beasley', 'PTS': 4.121, 'AST': 0.413, 'REB': 1.197},
 {'name': 'JaVale McGee', 'PTS': 11.787, 'AST': 0.987, 'REB': 6.294},
 {'name': 'Lance Stephenson', 'PTS': 7.505, 'AST': 1.775, 'REB': 3.324},
 {'name': 'Brandon Ingram', 'PTS': 15.827, 'AST': 2.338, 'REB': 4.207},
 {'name': 'Kyle Kuzma', 'PTS': 16.136, 'AST': 1.763, 'REB': 5.285},
 {'name': 'Svi Mykhailiuk', 'PTS': 2.335, 'AST': 0.891, 'REB': 0.507},
 {'name': 'Moritz Wagner', 'PTS': 0.0, 'AST': 0.987, 'REB': 0.362},
 {'name': 'Lonzo Ball', 'PTS': 7.897, 'AST': 4.432, 'REB': 5.509},
 {'name': 'Kentavious Caldwell-Pope',
  'PTS': 8.096,
  'AST': 0.607,
  'REB': 2.301},
 {'name': 'LeBron James', 'PTS': 28.154, 'AST': 6.496, 'REB': 7.79},
 {'name': 'Josh Hart', 'PTS': 8.946, 'AST': 1.276, 'REB': 3.334},
 {'name': 'Ivica Zubac', 'PTS': 2.209, 'AST': 0.283, 'REB': 3.462},
 {'name': 'Rajon Rondo', 'PTS': 8.119, 'AST': 6.323, 'REB': 4.514},
 {'name': '

In [17]:
for val in lal_json:
    print(val.keys())

dict_keys(['Tyson Chandler'])
dict_keys(['Michael Beasley'])
dict_keys(['JaVale McGee'])
dict_keys(['Lance Stephenson'])
dict_keys(['Brandon Ingram'])
dict_keys(['Kyle Kuzma'])
dict_keys(['Svi Mykhailiuk'])
dict_keys(['Moritz Wagner'])
dict_keys(['Lonzo Ball'])
dict_keys(['Kentavious Caldwell-Pope'])
dict_keys(['LeBron James'])
dict_keys(['Josh Hart'])
dict_keys(['Ivica Zubac'])
dict_keys(['Rajon Rondo'])
dict_keys(['Johnathan Williams'])
